# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [137]:
file_path_list = []
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data/'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [214]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [215]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [4]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [5]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [6]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

### Create tables.

`Table Name: song_playlist_session
column 1: SessionId
column 2: ItemInSession
column 3: Artist
column 4: Song
column 5: Length
PRIMARY KEY(sessionId, itemInSession)`


`Table Name: song_playlist_users
column 1: SessionId
column 2: UserId
column 3: ItemInSession
column 4: FirstName
column 5: LastName
column 6: Artist
column 7: Song
PRIMARY KEY((sessionId, userId),itemInSession)`


`Table Name: song_playlist_songs
column 1: Song
column 2: UserId
column 3: FirstName
column 4: LastName
PRIMARY KEY(song,userId)`

In [203]:
query = "CREATE TABLE IF NOT EXISTS song_playlist_session "
query = query + "(sessionId int, itemInSession int, artist text, song text, length double, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "CREATE TABLE IF NOT EXISTS song_playlist_users "
query = query + "(sessionId int, userId int, itemInSession int, firstName text,lastName text,artist text,song text, PRIMARY KEY ((sessionId,userId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "CREATE TABLE IF NOT EXISTS song_playlist_songs "
query = query + "(song text, userId int, firstName text,lastName text, PRIMARY KEY (song,userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Insert values.

In [204]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_playlist_session (sessionId,itemInSession,artist,song,length)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]),int(line[3]),line[0],line[9],float(line[5])))
        
        query1 = "INSERT INTO song_playlist_users (sessionId,userId,itemInSession,firstName,lastName,artist,song)"
        query1 = query1 + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query1, (int(line[8]),int(line[10]),int(line[3]),line[1],line[4],line[0],line[9]))
        
        query2 = "INSERT INTO song_playlist_songs (song,userId,firstName,lastName)"
        query2 = query2 + "VALUES (%s,%s,%s,%s)"
        session.execute(query2, (line[9],int(line[10]),line[1],line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [212]:
query = "select * from song_playlist_session limit 5"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
     print ( row.artist,row.song,row.length)

Regina Spektor The Calculation (Album Version) 191.08526
Octopus Project All Of The Champs That Ever Lived 250.95791
Tegan And Sara So Jealous 180.06159
Dragonette Okay Dolores 153.39057
Lil Wayne / Eminem Drop The World 229.58975


In [206]:
query = "select * from song_playlist_users limit 5"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
     print ( row.firstname,row.lastname,row.artist,row.song)

Jacob Klein Manowar Shell Shock
Jacob Klein Morcheeba Women Lose Weight (Feat: Slick Rick)
Jacob Klein Maroon 5 Won't Go Home Without You
Jacob Klein Train Hey_ Soul Sister
Jacob Klein LMFAO I'm In Miami Bitch


In [208]:
query = "select * from song_playlist_songs limit 5"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
     print ( row.firstname,row.lastname, row.song)

Chloe Cuevas Wonder What's Next
Chloe Cuevas In The Dragon's Den
Aleena Kirby Too Tough (1994 Digital Remaster)
Chloe Cuevas Rio De Janeiro Blue (Album Version)
Lily Koch My Place


## Create queries to ask the following three questions of the data

### 1.This query uses column 1 (SessionId) as the partition and column 2 (ItemInSession) is used to uniquely identify the rows within a partition.


In [209]:
query = "select artist, song, length from song_playlist_session WHERE sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print ( row.artist,row.song,row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### 2. This query uses column 1 (SessionId), column 2 (UserId) as the partition, and column 3 (ItemInSession) is used to uniquely identify the rows within a partition. The result is sorted in ascending order by ItemInSession.

In [210]:
query = "select firstName, lastName, artist, song from song_playlist_users WHERE sessionid = 182 and userid = 10 ORDER BY itemInSession"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print ( row.artist,row.song,row.firstname,row.lastname)
                    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz



### 3. This query uses column 1 (Session Id) and column 2 (UserId) is used to uniquely identify the rows within a partition.


In [211]:
query = "select firstName,lastName  from song_playlist_songs WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
     print ( row.firstname,row.lastname)
                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [199]:
query = "drop table song_playlist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table song_playlist_users"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table song_playlist_songs"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="unconfigured table song_playlist_users"
Error from server: code=2200 [Invalid query] message="unconfigured table song_playlist_songs"


### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()